In [195]:
import pandas as pd
import numpy as np
from functools import reduce
import math
path='/Users/sidsharma/Downloads/Master 50 State Data (Rev 2.0 for 2018-19) - Alaska.csv'
path2 ='/Users/sidsharma/Downloads/Cleaned - Master 50 State Data - Alaska.csv'
test = pd.read_csv(path)
test2 = pd.read_csv(path2)
pd.set_option('display.float_format', lambda x: '%.1f' % x)

unique1 = test.dropna(subset = ['NCES School ID']).copy(deep = True)
dict1 = unique1.set_index('School Name').to_dict()['NCES School ID']

def replace_NCES(df):
    if isinstance(df['NCES School ID'], str):
        return df['NCES School ID'] 
    else:
        if (math.isnan(df['NCES School ID'])):
            NCES = dict1.get(df['School Name'], None)
            return NCES
        else:
            return df['NCES School ID']

test['NCES School ID'] = test.apply(replace_NCES, axis = 1)
test2['NCES School ID'] = test2.apply(replace_NCES, axis = 1)

def standard(df):
    try: 
        temp = float(df['Duration'])
    except ValueError:
        temp = .5
    if (math.isnan(temp)):
        temp = .5
    if (temp>=.5 and (df['Grad'] == 'R' or df['Grad'] == 'Required') and (df['Type'] == 'S' or df['Type'] == 'Standalone')):
         return "Gold Standard"
    elif (temp>=.5 and (df['Grad'] == 'E' or df['Grad'] == 'Cluster') and (df['Type'] == 'S' or df['Type'] == 'Standalone')):
        return "Silver Standard"
    elif ((df['Grad'] == 'E' or df['Grad'] == 'Cluster') or (df['Grad'] == 'R' or df['Grad'] == 'Required')):
        return "Bronze Standard"
    else: 
        return "Incomplete Information"
    
def testaddcomma(series):
    return reduce(lambda x, y: str(x) + ', ' + str(y), series)

def highest_standard(item):
    if isinstance(item, str):
        if item.find('Gold Standard') != -1:
            return 'Gold Standard'
        elif item.find('Silver Standard') != -1:
            return 'Silver Standard'
        elif item.find('At least Bronze Standard') != -1:
            return 'At least Bronze Standard'
        elif item.find('Bronze Standard') != -1:
            return 'Bronze Standard'
    return 'Incomplete Information'

def dfinsert(df2):
    df2['Standard'] = df2[['Duration', 'Grad', 'Type', 'Class Name', 'School Name']].apply(standard, axis = 1)
    df2 = df2.apply(lambda x: x.strip() if isinstance (x, str) else x)
    df2 = df2.groupby(['NCES School ID', 'School Name']).agg({'Standard': testaddcomma})
    df2.reset_index(inplace = True)
    df2['Highest Standard'] = df2['Standard'].apply(highest_standard)
    return df2

test.rename(columns = lambda x: x.strip(), inplace = True)
test = dfinsert(test)
test.rename(index=str, columns={"Highest Standard": "Standards 2018"}, inplace = True)
test.reset_index(inplace = True)

test2.rename(columns = lambda x: x.strip(), inplace = True)
test2 = dfinsert(test2)
test2.rename(index=str, columns={"Highest Standard": "Standards 2017"}, inplace = True)
test2.reset_index(inplace = True)

test2['NCES School ID'] = test2['NCES School ID'].convert_objects(convert_numeric=True)
test2['NCES School ID'] = test2['NCES School ID'].astype('float64')
test['NCES School ID'] = test2['NCES School ID'].convert_objects(convert_numeric=True)
test['NCES School ID'] = test2['NCES School ID'].astype('float64')
test = test.merge(test2, left_on ='NCES School ID', right_on = 'NCES School ID', how = 'left')
columnstokeep = ['NCES School ID', 'School Name_x', 'Standards 2018', 'Standards 2017']
test = test[columnstokeep]
path3 = '/Users/sidsharma/Downloads/testtesttest.csv'
test.to_csv(path3)
print("2018 Standards\n", test['Standards 2018'].value_counts().sort_index())
print("2017 Standards\n", test['Standards 2017'].value_counts().sort_index())

2018 Standards
 Bronze Standard           55
Incomplete Information    34
Silver Standard           56
Name: Standards 2018, dtype: int64
2017 Standards
 Bronze Standard           44
Incomplete Information    58
Silver Standard           43
Name: Standards 2017, dtype: int64


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:76: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:78: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
